# Chap03 - 텐서플로의 기본 이해하기

텐서플로의 핵심 구축 및 동작원리를 이해하고, 그래프를 만들고 관리하는 방법과 상수, 플레이스홀더, 변수 등 텐서플로의 '구성 요소'에 대해 알아보자.

## 3.1 연산 그래프

### 3.1.1 연산 그래프란?

그래프는 아래의 그림과 같이 노드(node)나 꼭지점(vertex)로 연결 되어 있는 개체(entity)의 집합을 부르는 용어다. 노드들은 변(edge)을 통해 서로 연결되어 있다. 

![](./images/graph.png)

데이터 흐름 그래프에서(DataFlow Grapy)의 변(edge) 어떤 노드에서 다른 노드로 흘러가는(flow) 데이터의 방향을 정한다.

텐서플로에서 그래프의 각 **노드는 하나의 연산을 나타내며, 입력값을 받아 다른 노드로 전달할 결과값을 출력**한다.

### 3.1.2 연산 그래프의 장점

텐서플로는 그래프의 연결 상태를 기반으로 연산을 최적화한다. 각 그래프에는 노드 간에 의존관계(dependency)가 존재한다.. 예를 들어, 아래의 그림 'A'에서 노드 `e`는 노드 `c`에 **직접의존**(direct dependeny)하고 있고, 노드 `a`에는 **간접의존**(indirect dependency) 한다.

![](./images/graph02.png)

위의 그림에서 노드`e`를 계산하기 위해서는 노드 `c, b, a`만 계산 해주면 된다. 따라서, **의존관계를 이용해 연산량이 최소화**할 수 있다. 이처럼 그래프를 통해 각 노드의 모든 의존관계를 파악할 수 있다.

## 3.2 그래프, 세션, 페치

### 3.2.1 그래프 만들기

`import tensorflow as tf`를 통해 텐서플로를 import 하면 그 시점에 비어 있는 기본 그래프가 만들어지며, 우리가 만드는 모든 노드들은 이 기본 그래프에 자동으로 연결된다.

In [1]:
import tensorflow as tf

다음과 같이 간단한 6개의 노드를 만들어 보자. 먼저 `a, b, c` 노드에 `5, 2, 3`을 대입한다.

In [2]:
a = tf.constant(5)
b = tf.constant(2)
c = tf.constant(3)

다음 `d, e, f` 노드에는 `a, b, c`노드를 이용하여 간단한 연산을 수행한다.

In [3]:
d = tf.multiply(a, b)  # a * b
e = tf.add(c, b)  # c + b
f = tf.subtract(d, e)  # d - e

위에서 정의한 노드 및 연산을 그래프로 그려보면 아래와 같다.

<img src="./images/graph03.png" width="60%" height="60%" />

텐서플로에서는 위의 코드처럼 곱셈, 덧셈, 뺄셈을 텐서플로의 `tf.<operator>`를 사용하여 나타낼 수 있을 뿐만아니라 축약 연산자 즉, `*, +, -` 등을 사용할 수 있다.

| TensorFlow 연산      | 축약 연산자 | 설명                                                       |
| -------------------- | ----------- | ---------------------------------------------------------- |
| `tf.add()`           | `a + b`     | a와 b를 더함                                               |
| `tf.multiply()`      | `a * b`     | a와 b를 곱함                                               |
| `tf.subtract()`      | `a - b`     | a에서 b를 뺌                                               |
| `tf.divide()`        | `a / b`     | a를 b로 나눔                                               |
| `tf.pow()`           | `a ** b`    | $a^b$ 를 계산                                              |
| `tf.mod()`           | `a % b`     | a를 b로 나눈 나머지를 구함                                 |
| `tf.logical_and()`   | `a & b`     | a와 b의 논리곱을 구함. `dtype`은 반드시 `tf.bool`이어야 함 |
| `tf.greater()`       | `a > b`     | $a > b$ 의 True/False 값을 반환                            |
| `tf.greater_equal()` | `a >= b`    | $a \ge b$ 의 True/False 값을 반환                          |
| `tf.less_equal()`    | `a <= b`    | $ a \le b$ 의 True/False 값을 반환                         |
| `tf.less()`          | `a < b`     | $a < b$ 의 True/False 값을 반환                            |
| `tf.negative()`      | `-a`        | a의 반대 부호 값을 반환                                    |
| `tf.logical_not()`   | `~a`        | a의 반대의 참거짓을 반환. `tf.bool` 텐서만 적용 가능       |
| `tf.abs()`           | `abs(a)`    | a의 각 원소의 절대값을 반환                              |
| `tf.logical_or()`    | `a I b`     | a와 b의 논리합을 구함. `dtype`은 반드시 `tf.bool`이어야 함 |


### 3.2.2 세션을 만들고 실행하기

3.2.1에서 정의한 노드 및 연산 그래프를 실행하려면 아래의 코드 처럼 **세션(Session)**을 만들고 실행하면 된다.

In [4]:
import tensorflow as tf

# 노드 및 연산 그래프 정의
a = tf.constant(5)
b = tf.constant(2)
c = tf.constant(3)

d = tf.multiply(a, b)  # a * b
e = tf.add(c, b)  # c + b
f = tf.subtract(d, e)  # d - e

# 세션을 만들고 연산그래프 실행
sess = tf.Session()
outs = sess.run(f)
sess.close()
print("outs = {}".format(outs))

outs = 5


먼저, `tf.Session()`에서 그래프를 시작한다. `Session`객체는 파이썬 객체와 데이터, 객체의 메모리가 할당되어 있는 실행 환경 사이를 연결하며, 중간 결과를 저장하고 최종 결과를 작업 환경으로 보내준다. 위의 코드에서는 `Session` 객체를 `sess = tf.Session()` 에 정의했다. 

연산 그래프를 실행하려면 `Session`객체의 `run()` 메소드를 사용해야한다. 위의 코드에서 `sess.run(f)`는 아래의 그림처럼 출력이 나와야 하는 `f`노드에서 시작해서 역방향으로 의존관계에 따라 노드의 연산을 수행한다. 

<img src="./images/graph04.png" width="60%" height="60%" />

연산 수행이 완료되면 `sess.close()`를 통해 사용한 메모리를 해제하는 것이 좋다.

### 3.2.3 그래프의 생성과 관리

3.2.1에서 살펴 보았듯이, 텐서플로를 import 하면 바로 기본 그래프가 자동으로 만들어진다. 이 뿐만아니라 그래프를 추가로 생성하고 특정 연산의 관계를 직접 제어할 수도 있다. `tf.Graph()`는 텐서플로 객체로 표현되는 새로운 그래프를 만든다. 아래의 코드는 새로운 그래프를 만든 후 `g`에 할당한 코드이다.

In [5]:
import tensorflow as tf 

# 새 그래프(Graph) 생성
g = tf.Graph()

print('default graph :', tf.get_default_graph())  # default graph 
print('new graph :', g)  # new graph

a = tf.constant(5)  # a 노드 생성

print('a 노드가 g 그래프와 연결 되어 있나? :', a.graph is g)
print('a 노드가 기본 그래프와 연결 되어 있나? :', a.graph is tf.get_default_graph())

default graph : <tensorflow.python.framework.ops.Graph object at 0x0000025EEE9578D0>
new graph : <tensorflow.python.framework.ops.Graph object at 0x0000025EEE975898>
a 노드가 g 그래프와 연결 되어 있나? : False
a 노드가 기본 그래프와 연결 되어 있나? : True


위의 코드에서 보면 기본 그래프(`tf.get_default_graph()`)와 `g` 그래프(`g = tf.Graph()`)는 다른 텐서플로 객체임을 알 수 있다. 

또한, 노드 `a`에서 `a.graph`를 통해 `a`가 어떤 그래프에 연결되어 있는지 알 수 있다. 위의 코드에서는 새로운 그래프 `g`를 생성하였지만, 이 `g`그래프를 기본 그래프로 **지정** 해주지 않아 노드 `a`는 텐서플로를 import 하면서 생성된 기본 그래프에 연결 되어 있음을 알 수 있다.

#### `with` 구문을 사용한 그래프 연결하기

Python의 `with` 구문을 이용하면 원하는 그래프와 연결할 수 있다. `with`구문은 코드 실행이 **시작** 할 때 **설정**이 필요하고 코드가 종료 되는 시점에 **해제**가 필요한 경우에 사용하면 편리한 문법이다. 

이러한 `with` 구문에서 `as_default()` 메소드를 사용하면 해당 그래프를 기본 그래프로 지정해준다

In [6]:
import tensorflow as tf

g1 = tf.get_default_graph()
g2 = tf.Graph()

# g2 그래프가 기본 그래프인지 확인
print('g2가 기본 그래프인가? : ', g2 is tf.get_default_graph())

# with 구문을 이용한 
# g2를 기본 그래프로 지정하기
with g2.as_default():
    print('g2가 기본 그래프인가? : ', g2 is tf.get_default_graph())

# with 구문이 끝났으므로 
# g2를 기본 그래프에서 해제
print('g2가 기본 그래프인가? : ', g2 is tf.get_default_graph())

g2가 기본 그래프인가? :  False
g2가 기본 그래프인가? :  True
g2가 기본 그래프인가? :  False


### 3.2.4 페치(fetch)

3.2.2 예제에서 `sess.run(f)` 를 통해 `f`노드를 실행했다. 이처럼 `sess.run()`의 **인자**(parameter)인 `f`를 **페치(fetches)** 라고 하며, **연산하고자 하는 그래프의 요소에 해당**한다. 페치는 하나의 노드가 되거나 노드들로 이루어진 리스트(list)이다.  

In [7]:
with tf.Session() as sess:
    fetches = [a, b, c, d, e, f]
    outs = sess.run(fetches)
    
print("outs = {}".format(outs))
print(type(outs[0]))

outs = [5, 2, 3, 10, 5, 5]
<class 'numpy.int32'>


## 3.3 텐서의 흐름

이제 텐서플로에서 노드(node)와 엣지(edge)가 실제로 표현되는 방법 및 컨트롤하는 방법을 알아보도록 하자. 

### 3.3.1 노드는 연산, 엣지는 텐서 객체

앞의 예제에서 보았듯이, `tf.add(), tf.multiply()` 등으로 그래프에서 노드(node)를 만들 때, 실제로는 **연산 인스턴스가 생성**된다. 생성된 연산(인스턴스)들은 그래프가 실행되기 전까지는 연산한 값을 반환하지 않고, 계산된 결과를 다른 노드로 전달할 수 있는 핸들(handle), 즉 **흐름(flow)**으로 참조된다. 이러한 핸들은 그래프에서 엣지(edge)라고 할 수 있으며, 텐서 객체(Tensor object)라고 한다.

텐서플로는 모든 구성 요소가 담긴 그래프의 골격을 먼저 만들도록 설계되었다. 이 시점에는 실제 텐서(데이터)는 흐르지 않으며 연산 또한 수행되지 않는다. 세션(Session)이 실행되면 그래프에 텐서가 입력되고 연산이 수행된다.

<img src="./images/node_edge.png" width="75%" height="75%" />

아래의 예제코드는 세션이 실행되기 전과 실행된 후의 텐세객체를 출력한 예제이다. 아래의 출력결과에서 볼 수 있듯이, 텐서플로의 텐서 객체는 `name, shape, dtype` 속성이 있어 해당 객체의 특징을 확인할 수 있다.

In [14]:
tensor_a = tf.constant([[1, 2], [3, 4]])

print('Session이 실행되기 전 :', tensor_a)

sess = tf.Session()
out = sess.run(tensor_a)
print('Session이 실행된 후 :\n', out)

Session이 실행되기 전 : Tensor("Const_13:0", shape=(2, 2), dtype=int32)
Session이 실행된 후 :
 [[1 2]
 [3 4]]


### 3.3.2 데이터 타입

그래프를 통해 전달되는 데이터의 기본 단위는 숫자, 참거짓값(`True, False`), 스트링 요소들이다. 

<img src="./images/data_type.png" width="75%" height="75%" />

3.3.1의 예제에서 `tensor_a = tf.constant([[1, 2], [3, 4]])`는 데이터타입(`dtype`)을 정의하지 않았기 때문에 텐서플로가 자동으로 `int32`로 데이터 타입을 추측했다. 아래와 텐서 객체를 만들 때 데이터 타입을 정의해줄 수 있다.

In [15]:
tensor_a = tf.constant([[1, 2], [3, 4]], dtype=tf.float64)
print(tensor_a)
print(tensor_a.dtype)

Tensor("Const_14:0", shape=(2, 2), dtype=float64)
<dtype: 'float64'>


#### 형 변환 (Casting)

텐서플로에서 일치하지 않는 두 데이터 타입을 가지고 연산을 실행하면 예외가 발생하므로 그래프에서 데이터 타입이 일치하는지 확인하는 것이 중요하다. 아래의 표와 같이 텐서플로는 다양한 데이터타입을 지원한다.

| 데이터 타입 이름 | 파이썬 데이터 타입 | 설명                                                         |
| :--------------- | :----------------- | :----------------------------------------------------------- |
| DT_FLOAT         | `tf.float32`       | 32비트 부동소수점 숫자                                       |
| DT_DOUBLE        | `tf.float64`       | 64비트 부동소수점 숫자                                       |
| DT_INT8          | `tf.int8`          | 8비트 정수                                                   |
| DT_INT16         | `tf.int16`         | 16비트 정수                                                  |
| DT_INT32         | `tf.int32`         | 32비트 정수                                                  |
| DT_INT64         | `tf.int64`         | 64비트 정수                                                  |
| DT_UINT8         | `tf.uint8`         | 8비트 부호 없는 정수                                         |
| DT_UINT16        | `tf.uint16`        | 16비트 부호 없는 정수                                        |
| DT_STRING        | `tf.string`        | 가변 길이 바이트 배열이며 텐서의 각 요소는 바이트의 배열     |
| DT_BOOL          | `tf.bool`          | 참거짓값                                                     |
| DT_COMPLEX64     | `tf.complex64`     | 2개의 32비트 부동소수점 숫자로 구성된 복소수로 각각 실수부와 허수부 |
| DT_COMPLEX128    | `tf.complex128`    | 2개의 64비트 부동소수점 숫자로 구성된 복소수로 각각 실수부와 허수부 |
| DT_QINT8         | `tf.qint8`         | 양자화 연산(quantized operation)에 사용되는 8비트 정수       |
| DT_QINT32        | `tf.qint32`        | 양자화 연산에 사용되는 32비트 정수                           |
| DT_QUINT8        | `tf.quint8`        | 양자화 연산에 사용되는 8비트 부호 없는 정수                  |

### 3.3.3 텐서 배열과 형태

텐서플로에서 텐서(Tensor)는 다음과 같이 두 가지 의미로 볼 수 있다.

- 그래프에서 연산의 결과, 파이썬 API에서 사용하는 객체의 이름
- $n$차원 배열을 가리키는 수학 용어. 
    - $1 \times 1$ 텐서는 스칼라, $1 \times n$ 텐서는 벡터, $n \times n$ 텐서는 행렬, $n \times n \times n$ 텐서는 3차원 배열
    - 텐서플로에서는 다차원 배열, 벡터, 행렬, 스칼라 등을 그래프에서 전달되는 모든 데이터를 **텐서**로 간주한다.

파이썬의 자료형인 리스트(list)나 NumPy의 배열을 사용하여 텐서를 초기화할 수 있다. 아래의 예제는 이 두가지를 사용하여 텐서를 초기화 하는 예제다. 

In [20]:
import numpy as np
import tensorflow as tf

c = tf.constant([[1, 2, 3], 
                 [4, 5, 6]])
print('Python List input :', c.get_shape())

c = tf.constant(np.array([
                  [[1, 2, 3], 
                   [4, 5, 6]],
                    
                  [[1, 1, 1], 
                   [2, 2, 2]]
                ]))

print("3d NumPy array input :", c.get_shape())

Python List input : (2, 3)
3d NumPy array input : (2, 2, 3)
